# Yanoya Data

## Data

### Data Source Python

In [0]:
import json
import altair as alt
import numpy as np
import pandas as pd

yny_data_json = '''
{
    "top": {
        "atomic_bool": false,
        "atomic_int": 1,
        "atomic_float": 0.11,
        "atomic_str": "abc1",
        "mutable_str": "abc2",
        "immutable_str": "constant",
        "child": {
            "another_atomic_bool": true,
            "another_atomic_int": 2,
            "another_atomic_float": -0.22,
            "another_atomic_str": "abc3",
            "child": {
                "name": "abc4",
                "age": 3
            },
            "child_optional": null,
            "sum": "other",
            "children": [
                {
                    "name": "abc5",
                    "age": 4
                },
                {
                    "name": "abc6",
                    "age": 5
                },
                {
                    "name": "abc7",
                    "age": 6
                }
            ],
            "dict_child_bool": {}
        },
        "child_optional": {
            "another_atomic_bool": false,
            "another_atomic_int": 7,
            "another_atomic_float": 0.33,
            "another_atomic_str": "abc8",
            "child": {
                "name": "abc9",
                "age": 8
            },
            "child_optional": null,
            "sum": {
                "name": "abc10",
                "age": 9
            },
            "children": [
                {
                    "name": "abc11",
                    "age": 10
                },
                {
                    "name": "abc12",
                    "age": 11
                },
                {
                    "name": "abc13",
                    "age": 12
                }
            ],
            "dict_child_bool": {}
        },
        "enum": "this",
        "enum_list": [
            "this",
            "that",
            "either or"
        ],
        "children": [
            {
                "another_atomic_bool": true,
                "another_atomic_int": 13,
                "another_atomic_float": -0.44,
                "another_atomic_str": "abc14",
                "child": {
                    "name": "abc15",
                    "age": 14
                },
                "child_optional": null,
                "sum": "other",
                "children": [
                    {
                        "name": "abc16",
                        "age": 15
                    },
                    {
                        "name": "abc17",
                        "age": 16
                    },
                    {
                        "name": "abc18",
                        "age": 17
                    }
                ],
                "dict_child_bool": {}
            },
            {
                "another_atomic_bool": false,
                "another_atomic_int": 18,
                "another_atomic_float": 0.55,
                "another_atomic_str": "abc19",
                "child": {
                    "name": "abc20",
                    "age": 19
                },
                "child_optional": null,
                "sum": {
                    "name": "abc21",
                    "age": 20
                },
                "children": [
                    {
                        "name": "abc22",
                        "age": 21
                    },
                    {
                        "name": "abc23",
                        "age": 22
                    },
                    {
                        "name": "abc24",
                        "age": 23
                    }
                ],
                "dict_child_bool": {}
            },
            {
                "another_atomic_bool": true,
                "another_atomic_int": 24,
                "another_atomic_float": -0.66,
                "another_atomic_str": "abc25",
                "child": {
                    "name": "abc26",
                    "age": 25
                },
                "child_optional": null,
                "sum": "other",
                "children": [
                    {
                        "name": "abc27",
                        "age": 26
                    },
                    {
                        "name": "abc28",
                        "age": 27
                    },
                    {
                        "name": "abc29",
                        "age": 28
                    }
                ],
                "dict_child_bool": {}
            }
        ],
        "dict_str_child": {
            "abc30": {
                "another_atomic_bool": false,
                "another_atomic_int": 29,
                "another_atomic_float": 0.77,
                "another_atomic_str": "abc31",
                "child": {
                    "name": "abc32",
                    "age": 30
                },
                "child_optional": null,
                "sum": {
                    "name": "abc33",
                    "age": 31
                },
                "children": [
                    {
                        "name": "abc34",
                        "age": 32
                    },
                    {
                        "name": "abc35",
                        "age": 33
                    },
                    {
                        "name": "abc36",
                        "age": 34
                    }
                ],
                "dict_child_bool": {}
            }
        }
    }
}
'''

yny_data_py = json.loads(yny_data_json)
yny_item_name = list(yny_data_py)[0]
yny_data_without_root_py = yny_data_py[yny_item_name]
yny_data_py

### Helpers

In [0]:
def yny_get(data, path):
    path_parts = path.split("/")
    def yny_path_rec(data, path_parts):
        if not path_parts:
            return data
        if isinstance(data, dict):
            if path_parts[0] in data:
                return yny_path_rec(data[path_parts[0]], path_parts[1:])
        if isinstance(data, list):
            if len(data)==1:
                return yny_path_rec(data[0], path_parts[1:])
        return None
    return yny_path_rec(data, path_parts)

def yny_get_list(data, path):
    path_parts = path.split("/")
    def yny_path_rec(data, path_parts, ret):
        if not path_parts:
            if isinstance(data, list):
                ret.extend(data)
            else:
                ret.append(data)
            return
        if isinstance(data, dict):
            if path_parts[0] in data:
                return yny_path_rec(data[path_parts[0]], path_parts[1:], ret)
        if isinstance(data, list) and path_parts[0]=="*":
            for el in data:
                yny_path_rec(el, path_parts[1:], ret)
        return None
    ret = []
    yny_path_rec(data, path_parts, ret)
    return ret

### JSON Format

In [0]:
json.dumps(yny_data_py)

### YAML Format

In [0]:
import yaml
yny_data_yaml = yaml.dump(yny_data_py)
print(yny_data_yaml,end=chr(10))

### Pandas Dataframe

In [0]:
yny_data_df = pd.DataFrame({
  'another_atomic_bool': yny_get_list(yny_data_without_root_py, 'children/*/another_atomic_bool'),
  'another_atomic_int': yny_get_list(yny_data_without_root_py, 'children/*/another_atomic_int'),
  'child/age': yny_get_list(yny_data_without_root_py, 'children/*/child/age'),
  'another_atomic_float': yny_get_list(yny_data_without_root_py, 'children/*/another_atomic_float'),
  'another_atomic_str': yny_get_list(yny_data_without_root_py, 'children/*/another_atomic_str'),
  'child/name': yny_get_list(yny_data_without_root_py, 'children/*/child/name'),
  'sum': yny_get_list(yny_data_without_root_py, 'children/*/sum')
  
})
yny_data_df

### CSV Format

In [0]:
print(yny_data_df.to_csv(index=False),end=chr(10))

### Line Chart

In [0]:
chart = alt.Chart(yny_data_df).mark_line().encode(x='another_atomic_int',y='child/age')
chart

### Pie Chart

In [0]:
chart = alt.Chart(yny_data_df).mark_arc().encode(
 theta="another_atomic_int",
 color="child/age"
)
chart